# Boosting

A metodologia foi inicialmente criada para resolver uma classificação, a idéia principal por é encontrar hipóteses fracas, aprender repetidamente e combinar essas hipóteses fracas dentro de uma única hipótese.
  
É um método de ensemble? Sim.  
**Métodos de ensemble** que tem como objetivo **combinar as predições de diversos estimadores mais simples** para gerar uma **predição final mais robusta**

- **Métodos de boosting**: têm como procedimento geral a construção de estimadores de forma sequencial, de modo que estimadores posteriores tentam reduzir o **viés** do estimador conjunto, que leva em consideração estimadores anteriores. Ex.: **adaboost**.

## Métodos de Ensemble


Há uma classe de algoritmos de Machine Learning, os chamados **métodos de ensemble** que tem como objetivo **combinar as predições de diversos estimadores mais simples** para gerar uma **predição final mais robusta**

Os métodos de ensemble costuman ser divididos em duas classes:

- **Métodos de média**: têm como procedimento geral construir diversos estimadores independentes, e tomar a média de suas predições como a predição final. O principal objetivo do método é reduzir **variância**, de modo que o modelo final seja melhor que todos os modelos individuais. Ex.: **random forest.**
<br>

- **Métodos de boosting**: têm como procedimento geral a construção de estimadores de forma sequencial, de modo que estimadores posteriores tentam reduzir o **viés** do estimador conjunto, que leva em consideração estimadores anteriores. Ex.: **adaboost**.

Há, ainda, uma terceira classe de método de ensemble, o chamado [stacking ensemble](https://machinelearningmastery.com/stacking-ensemble-machine-learning-with-python/), que consiste em "empilhar" modelos de modo a produzir a mistura. Não veremos esta modalidade em detalhes, mas deixo como sugestão para estudos posteriores! :)

Para mais detalhes sobre métodos de ensemble no contexto do sklearn, [clique aqui!](https://scikit-learn.org/stable/modules/ensemble.html)

Na aula de hoje, vamos conhecer em detalhes os procedimentos de bagging e boosting, ilustrados pelos métodos Random Forest e AdaBoost, respectivamente. Vamos lá!

______

### Bagging vs Boosting

Pra lembrar as principais diferenças entre os dois métodos de ensemble que estudamos:

<img src=https://pluralsight2.imgix.net/guides/81232a78-2e99-4ccc-ba8e-8cd873625fdf_2.jpg width=600>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

____
____
____

## Boosting & AdaBoost

O AdaBoost significa **Adaptive Boosting**, e tem como procedimento geral **a criação sucessiva dos chamados weak learners**, que são modelos bem fracos de aprendizagem - geralmente, **árvores de um único nó (stumps)**.

<img src="https://miro.medium.com/max/1744/1*nJ5VrsiS1yaOR77d4h8gyw.png" width=300>

O AdaBoost utiliza os **erros da árvore anterior para melhorar a próxima árvore**. As predições finais são feitas com base **nos pesos de cada stump**, cuja determinação faz parte do algoritmo!

<img src="https://static.packt-cdn.com/products/9781788295758/graphics/image_04_046-1.png" width=700>

Vamos entender um pouco melhor...

Aqui, o bootstrapping não é utilizado: o método começa treinando um classificador fraco **no dataset original**, e depois treina diversas cópias adicionais do classificador **no mesmo dataset**, mas dando **um peso maior às observações que foram classificadas erroneamente** (ou, no caso de regressões, a observações **com o maior erro**).

Assim, após diversas iterações, classificadores/regressores vão sequencialmente "focando nos casos mais difíceis", e construindo um classificador encadeado que seja forte, apesar de utilizar diversos classificadores fracos em como elementos fundamentais.

<img src="https://www.researchgate.net/profile/Zhuo_Wang8/publication/288699540/figure/fig9/AS:668373486686246@1536364065786/Illustration-of-AdaBoost-algorithm-for-creating-a-strong-classifier-based-on-multiple.png" width=500>


De forma resumida, as principais ideias por trás deste algoritmo são:

- O algoritmo cria e combina um conjunto de **modelos fracos** (em geral, stumps);
- Cada stump é criado **levando em consideração os erros do stump anterior**;
- Alguns dos stumps têm **maior peso de decisão** do que outros na predição final;

As classes no sklearn são:

- [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)

- [AdaBoostRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor)

Note que não há muitos hiperparâmetros. O mais importante, que deve ser tunado com o grid/random search, é:

- `n_estimators` : o número de weak learners encadeados;

Além disso, pode também ser interessante tunar os hiperparâmetros dos weak learners. Isso é possível de ser feito, como veremos a seguir!


Uma animação para entendermos melhor...  
- O projeto https://periodicos.uff.br/anaisdoser/article/download/29032/16865/100072
- O resultado https://mateusmaia.shinyapps.io/adaboosting/

Primeiro, vamos começar com nosso baseline:

In [20]:
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer

from sklearn.metrics import classification_report

from sklearn import set_config
set_config(display="diagram")

In [3]:
df = pd.read_csv('dados/german_credit_data.csv', index_col=0)
df.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad


In [4]:
df['Risk'] = df['Risk'].replace({'good': 1, 'bad': 0})

In [5]:
df.isna().sum()

Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
Risk                  0
dtype: int64

In [6]:
df.describe()

,Age,Job,Credit amount,Duration,Risk
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,35.546000,1.904000,3271.258000,20.903000,0.700000
std,11.375469,0.653614,2822.736876,12.058814,0.458487
min,19.000000,0.000000,250.000000,4.000000,0.000000
25%,27.000000,2.000000,1365.500000,12.000000,0.000000
50%,33.000000,2.000000,2319.500000,18.000000,1.000000
75%,42.000000,2.000000,3972.250000,24.000000,1.000000
max,75.000000,3.000000,18424.000000,72.000000,1.000000


In [8]:
df['Job'].value_counts()

2    630
1    200
3    148
0     22
Name: Job, dtype: int64

In [9]:
target = 'Risk'

feat_num = ['Age', 'Credit amount', 'Duration']

feat_cat_or = ['Job', 'Saving accounts', 'Checking account']
feat_cat_no = ['Sex', 'Housing', 'Purpose']

In [10]:
df['Checking account'].unique()

array(['little', 'moderate', nan, 'rich'], dtype=object)

In [11]:
feat_cat_or_seq = {
    'Job': [0, 1, 2, 3],
    'Saving accounts': ['little', 'moderate', 'quite rich', 'rich'],
    'Checking account': ['little', 'moderate', 'rich']
}

In [12]:
def cria_seq_replace(lista):
    return dict([(valor, i) for i, valor in enumerate(feat_cat_or_seq[lista])])

In [13]:
cria_seq_replace('Saving accounts')

{'little': 0, 'moderate': 1, 'quite rich': 2, 'rich': 3}

In [14]:
for feature in feat_cat_or:
    df[feature] = df[feature].replace(cria_seq_replace(feature))

In [15]:
df.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,0.0,1169,6,radio/TV,1
1,22,female,2,own,0.0,1.0,5951,48,radio/TV,0
2,49,male,1,own,0.0,NaN,2096,12,education,1
3,45,male,2,free,0.0,0.0,7882,42,furniture/equipment,1
4,53,male,2,free,0.0,0.0,4870,24,car,0


In [16]:
X = df.drop(columns=target)
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, 
                                                    random_state=42, stratify=y)

#### Pipeline Customizado
https://towardsdatascience.com/pipelines-custom-transformers-in-scikit-learn-the-step-by-step-guide-with-python-code-4a7d9b068156

In [17]:
def cria_pipe_pre():
    pipe_pre_num = Pipeline([
        ('pre_inp_num', SimpleImputer(strategy='mean')),
        ('pre_sc_std', StandardScaler())
    ])

    pipe_pre_cat_or = Pipeline([
        ('pre_inp_cat_or', SimpleImputer(strategy='constant', fill_value=-1)),
        ('pre_sc_std', StandardScaler())
    ])

    pipe_pre_cat_no = Pipeline([
        ('pre_inp_cat_no', SimpleImputer(strategy='most_frequent')),
        ('pre_tra_one', OneHotEncoder(drop='first'))
    ])

    pipe_pre = ColumnTransformer([
        ('pipe_pre_num', pipe_pre_num, feat_num),
        ('pipe_pre_cat_or', pipe_pre_cat_or, feat_cat_or),
        ('pipe_pre_cat_no', pipe_pre_cat_no, feat_cat_no)
    ])
    
    return pipe_pre

In [18]:
pipe_pre = cria_pipe_pre()

In [21]:
pipe_pre.fit(X_train)

ColumnTransformer(transformers=[('pipe_pre_num',
                                 Pipeline(steps=[('pre_inp_num',
                                                  SimpleImputer()),
                                                 ('pre_sc_std',
                                                  StandardScaler())]),
                                 ['Age', 'Credit amount', 'Duration']),
                                ('pipe_pre_cat_or',
                                 Pipeline(steps=[('pre_inp_cat_or',
                                                  SimpleImputer(fill_value=-1,
                                                                strategy='constant')),
                                                 ('pre_sc_std',
                                                  StandardScaler())]),
                                 ['Job', 'Saving accounts',
                                  'Checking account']),
                                ('pipe_pre_cat_no',
                                 Pipeline(steps=[('pre_inp_cat_no',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('pre_tra_one',
                                                  OneHotEncoder(drop='first'))]),
                                 ['Sex', 'Housing', 'Purpose'])])

In [22]:
from sklearn.ensemble import AdaBoostClassifier

pipe_ab= Pipeline([
    ('pipe_pre', pipe_pre),
    ('est_ab', AdaBoostClassifier(random_state=42))
])

In [23]:
pipe_ab.fit(X_train, y_train)

Pipeline(steps=[('pipe_pre',
                 ColumnTransformer(transformers=[('pipe_pre_num',
                                                  Pipeline(steps=[('pre_inp_num',
                                                                   SimpleImputer()),
                                                                  ('pre_sc_std',
                                                                   StandardScaler())]),
                                                  ['Age', 'Credit amount',
                                                   'Duration']),
                                                 ('pipe_pre_cat_or',
                                                  Pipeline(steps=[('pre_inp_cat_or',
                                                                   SimpleImputer(fill_value=-1,
                                                                                 strategy='constant')),
                                                                  ('pre_sc_std',
                                                                   StandardScaler())]),
                                                  ['Job', 'Saving accounts',
                                                   'Checking account']),
                                                 ('pipe_pre_cat_no',
                                                  Pipeline(steps=[('pre_inp_cat_no',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('pre_tra_one',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['Sex', 'Housing',
                                                   'Purpose'])])),
                ('est_ab', AdaBoostClassifier(random_state=42))])

In [28]:
y_pred = pipe_ab.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.60      0.45      0.51        60
           1       0.79      0.87      0.83       140

    accuracy                           0.74       200
   macro avg       0.69      0.66      0.67       200
weighted avg       0.73      0.74      0.73       200



In [29]:
pipe_ab['est_ab'].estimators_

[DecisionTreeClassifier(max_depth=1, random_state=1608637542),
 DecisionTreeClassifier(max_depth=1, random_state=1273642419),
 DecisionTreeClassifier(max_depth=1, random_state=1935803228),
 DecisionTreeClassifier(max_depth=1, random_state=787846414),
 DecisionTreeClassifier(max_depth=1, random_state=996406378),
 DecisionTreeClassifier(max_depth=1, random_state=1201263687),
 DecisionTreeClassifier(max_depth=1, random_state=423734972),
 DecisionTreeClassifier(max_depth=1, random_state=415968276),
 DecisionTreeClassifier(max_depth=1, random_state=670094950),
 DecisionTreeClassifier(max_depth=1, random_state=1914837113),
 DecisionTreeClassifier(max_depth=1, random_state=669991378),
 DecisionTreeClassifier(max_depth=1, random_state=429389014),
 DecisionTreeClassifier(max_depth=1, random_state=249467210),
 DecisionTreeClassifier(max_depth=1, random_state=1972458954),
 DecisionTreeClassifier(max_depth=1, random_state=1572714583),
 DecisionTreeClassifier(max_depth=1, random_state=1433267572),


Vamos deixar o base_estimator explícito

In [30]:
from sklearn.tree import DecisionTreeClassifier

In [31]:
basal = DecisionTreeClassifier(max_depth=2)

pipe_ab = Pipeline([
    ('pipe_pre', cria_pipe_pre()),
    ('est_ab', AdaBoostClassifier(base_estimator=basal, random_state=42))
])

pipe_ab.fit(X_train, y_train)

y_pred = pipe_ab.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.52      0.48      0.50        60
           1       0.78      0.81      0.80       140

    accuracy                           0.71       200
   macro avg       0.65      0.65      0.65       200
weighted avg       0.70      0.71      0.71       200



Podemos, também, mudar o estimador basal. Por exemplo, uma regressão logística fortemente regularizada.

In [32]:
from sklearn.linear_model import LogisticRegression

In [33]:
basal = LogisticRegression(C=0.1, random_state=42)

pipe_ab = Pipeline([
    ('pipe_pre', cria_pipe_pre()),
    ('est_ab', AdaBoostClassifier(base_estimator=basal, random_state=42))
])

pipe_ab.fit(X_train, y_train)

y_pred = pipe_ab.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.44      0.13      0.21        60
           1       0.71      0.93      0.81       140

    accuracy                           0.69       200
   macro avg       0.58      0.53      0.51       200
weighted avg       0.63      0.69      0.63       200



Não ficou muito legal. Por isso que, apesar de ser possível usar outros estimadores basais, é comum usarmos stumps mesmo (árvores com uma única quebra).

Vamos agora fazer o gridsearch!

In [34]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [37]:
basal = DecisionTreeClassifier(max_depth=1)

pipe_ab = Pipeline([
    ('pipe_pre', cria_pipe_pre()),
    ('est_ab', AdaBoostClassifier(base_estimator=basal, random_state=42))
])

espaco_hiper = {
    'est_ab__learning_rate': [0.01, 0.015, 0.1, 0.15, 0.5],
    'est_ab__n_estimators': [10, 50, 100, 200, 400, 800, 1600],
    'est_ab__base_estimator__criterion': ['gini', 'entropy']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_ab = GridSearchCV(
    estimator=pipe_ab,
    param_grid=espaco_hiper,
    scoring='f1',
    cv=cv,
    verbose=0,
    n_jobs=31
)

grid_ab.fit(X_train, y_train)

y_pred = grid_ab.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.65      0.40      0.49        60
           1       0.78      0.91      0.84       140

    accuracy                           0.76       200
   macro avg       0.71      0.65      0.67       200
weighted avg       0.74      0.76      0.74       200



In [38]:
grid_ab.best_estimator_

Pipeline(steps=[('pipe_pre',
                 ColumnTransformer(transformers=[('pipe_pre_num',
                                                  Pipeline(steps=[('pre_inp_num',
                                                                   SimpleImputer()),
                                                                  ('pre_sc_std',
                                                                   StandardScaler())]),
                                                  ['Age', 'Credit amount',
                                                   'Duration']),
                                                 ('pipe_pre_cat_or',
                                                  Pipeline(steps=[('pre_inp_cat_or',
                                                                   SimpleImputer(fill_value=-1,
                                                                                 strategy='constant')),
                                                                  ('pre_sc_std',
                                                                   StandardScaler())]),
                                                  ['Job', 'Saving accounts',
                                                   'Checking account']),
                                                 ('pipe_pre_cat_no',
                                                  Pipeline(steps=[('pre_inp_cat_no',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('pre_tra_one',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['Sex', 'Housing',
                                                   'Purpose'])])),
                ('est_ab',
                 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                                    learning_rate=0.015, n_estimators=1600,
                                    random_state=42))])

In [39]:
grid_ab.best_params_

{'est_ab__base_estimator__criterion': 'gini',
 'est_ab__learning_rate': 0.015,
 'est_ab__n_estimators': 1600}

_________
_______
_________

##  Gradient boosting

Além dos métodos que estudamos, há ainda outras classes de métodos de ensemble!

Em particular, a classe de modelos que se utilizam do procedimento de **gradient boosting**.

O gradient boosting também é baseado no princípio de boosting (utilização de weak learners sequencialmente adicionados de modo a **sequencialmente minimizar os erros cometidos**).

<img src=https://miro.medium.com/max/788/1*pEu2LNmxf9ttXHIALPcEBw.png width=600>

Mas este método implementa o boosting através de um **gradiente** explícito.

A ideia é que caminhemos na direção do **erro mínimo** de maneira iterativa **passo a passo**.

Este caminho se dá justamente pelo **gradiente** da **função de custo/perda**, que mede justamente os erros cometidos.

<img src=https://upload.wikimedia.org/wikipedia/commons/a/a3/Gradient_descent.gif width=400>

Este método é conhecido como:

### Gradiente descendente

Deixei em ênfase porque este será um método de **enorme importância** no estudo de redes neurais (e é, em geral, um método de otimização muito utilizado).

O objetivo geral do método é bem simples: determinar quais são os **parâmetros** da hipótese que minimizam a função de custo/perda. Para isso, o método "percorre" a função de erro, indo em direção ao seu mínimo (e este "caminho" feito na função se dá justamente pela **determinação iterativa dos parâmetros**, isto é, **a cada passo, chegamos mais perto dos parâmetros finais da hipótese**, conforme eles são ajustados aos dados.

> **Pequeno interlúdio matemático:** o gradiente descendente implementado pelo gradient boosting é, na verdade, um **gradiente descendente funcional**, isto é, desejamos encontrar não um conjunto de parâmetros que minimiza o erro, mas sim **introduzir sequencialmente weak learners (hipótese simples) que minimizam o erro**. Desta forma, o gradient boosting minimiza a função de custo ao ecolher iterativamente hipóteses simples que apontam na direção do mínimo, neste espaço funcional.

Apesar do interlúdio acima, não precisamos nos preocupar muito com os detalhes matemáticos: o que importa é entender que no caso do gradient boosting, há alguns pontos importantes:

- Uma **função de custo/perda (loss)** é explicitamente minimizada por um procedimento de gradiente;

- O gradiente está relacionado com o procedimento de **encadeamento progressivo entre weak learners**, seguindo a ideia do boosting.

Pra quem quiser saber um pouco mais de detalhes (e se aventurar na matemática), sugiro [este post](https://www.gormanalysis.com/blog/gradient-boosting-explained/) ou então [este site](https://explained.ai/gradient-boosting/), que contém vários materiais ótimos para entender o método com todos os detalhes matemáticos.

Os [vídeos do StatQuest](https://www.youtube.com/playlist?list=PLblh5JKOoLUJjeXUvUE0maghNuY2_5fY6) também são uma boa referência!

As classes do sklearn são:

- [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)

- [GradientBoostingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor)

E os principais hiperparâmetros a serem ajustados são:

- `n_estimators` : novamente, o número de weak learners encadeados.

- `learning_rate` : a constante que multiplica o gradiente no gradiente descendente. Essencialmente, controla o "tamanho do passo" a ser dado em direção ao mínimo.

Segundo o próprio [User Guide](https://scikit-learn.org/stable/modules/ensemble.html#gradient-boosting): "*Empirical evidence suggests that small values of `learning_rate` favor better test error. The lireature recommends to set the learning rate to a small constant (e.g. `learning_rate <= 0.1`) and choose `n_estimators` by early stopping.*"

Ainda sobre a learning rate, as ilustrações a seguir ajudam a entender sua importância:

<img src=https://www.jeremyjordan.me/content/images/2018/02/Screen-Shot-2018-02-24-at-11.47.09-AM.png width=700>

<img src=https://cdn-images-1.medium.com/max/1440/0*A351v9EkS6Ps2zIg.gif width=500>

In [44]:
from sklearn.ensemble import GradientBoostingClassifier

In [45]:
pipe_gb = Pipeline([
    ('pipe_pre', cria_pipe_pre()),
    ('est_gb', GradientBoostingClassifier(learning_rate=0.1, 
                                          n_estimators=80, random_state=42))
])
    
pipe_gb.fit(X_train, y_train)

y_pred = pipe_gb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.40      0.51        60
           1       0.78      0.93      0.85       140

    accuracy                           0.77       200
   macro avg       0.74      0.66      0.68       200
weighted avg       0.76      0.77      0.75       200



Pra casa: grid search para otimizar os hiperparâmetros!